In [6]:
from transformers import pipeline
from nltk import sent_tokenize
import nltk
import torch
from glob import glob


In [3]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# Load Model

In [4]:
model_name = "facebook/bart-large-mnli"


In [2]:
def load_model(device=-1):
    print("Device set to use", "CPU" if device == -1 else "GPU")
    theme_classifier = pipeline(
        "zero-shot-classification",
        model="facebook/bart-large-mnli",
        device=device
    )
    return theme_classifier

theme_classifier = load_model()  # ✅ No errors now

   

Device set to use CPU


Device set to use cpu


In [25]:
import sys
print(sys.executable)


d:\Tv series analysis\.venv\Scripts\python.exe


In [1]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1)
print(classifier("I love space adventures", candidate_labels=["sci-fi", "romance", "comedy"]))


d:\Tv series analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
d:\Tv series analysis\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HOME\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more det

{'sequence': 'I love space adventures', 'labels': ['sci-fi', 'comedy', 'romance'], 'scores': [0.9929682612419128, 0.004026405978947878, 0.0030053311493247747]}


In [3]:
theme_list = ["friendship","hope","sacrifice","battle","self development","betrayal","love","dialogue"]


In [4]:
theme_classifier(
    "I gave him a right hook then a left jab",
    theme_list,
    multi_label=True
)

{'sequence': 'I gave him a right hook then a left jab',
 'labels': ['battle',
  'self development',
  'hope',
  'sacrifice',
  'dialogue',
  'betrayal',
  'love',
  'friendship'],
 'scores': [0.9121254682540894,
  0.475002646446228,
  0.08781813830137253,
  0.045000508427619934,
  0.020133016631007195,
  0.012040519155561924,
  0.004292406141757965,
  0.0028172244783490896]}

# Load Dataset


In [7]:
files = glob('../data/Subtitles/*.ass')

In [8]:
files[:5]


['../data/Subtitles\\Naruto Season 1 - 01.ass',
 '../data/Subtitles\\Naruto Season 1 - 02.ass',
 '../data/Subtitles\\Naruto Season 1 - 03.ass',
 '../data/Subtitles\\Naruto Season 1 - 04.ass',
 '../data/Subtitles\\Naruto Season 1 - 05.ass']

In [9]:
with open(files[0],'r') as file:
    lines = file.readlines()
    lines = lines[27:]
    lines =  [ ",".join(line.split(',')[9:])  for line in lines ]

In [10]:
lines[:2]


['A long time ago, a powerful demon fox\\Nappeared with nine tails.\n',
 'With its powerful tails,\n']

In [11]:
lines = [ line.replace('\\N',' ') for line in lines]


In [12]:
lines[:2]


['A long time ago, a powerful demon fox appeared with nine tails.\n',
 'With its powerful tails,\n']

In [13]:
" ".join(lines[:10])


"A long time ago, a powerful demon fox appeared with nine tails.\n With its powerful tails,\n it could smash mountains and create tidal waves.\n A band of Ninjas rose to defend their village from attack.\n We have to wait until the Fourth Hokage gets here!\n We can't let it get any closer to our village!\n One great Ninja was able to imprison the monster,\n but died in the process.\n This Ninja was known asâ€¦ the Fourth Hokage.\n Naruto!\n"

In [14]:
int(files[0].split('-')[-1].split('.')[0].strip())


1

In [15]:
def load_subtitles_dataset(dataset_path):
    subtitles_paths = glob(dataset_path+'/*.ass')

    scripts=[]
    episode_num=[]

    for path in subtitles_paths:

        #Read Lines
        with open(path,'r') as file:
            lines = file.readlines()
            lines = lines[27:]
            lines =  [ ",".join(line.split(',')[9:])  for line in lines ]
        
        lines = [ line.replace('\\N',' ') for line in lines]
        script = " ".join(lines)

        episode = int(path.split('-')[-1].split('.')[0].strip())

        scripts.append(script)
        episode_num.append(episode)

    df = pd.DataFrame.from_dict({"episode":episode_num, "script":scripts })
    return df